In [ ]:
pip install sentencepiece transformers contractions emoji

     |████████████████████████████████| 1.2 MB 5.5 MB/s 
     |████████████████████████████████| 3.1 MB 27.5 MB/s 
     |████████████████████████████████| 170 kB 48.3 MB/s 
     |████████████████████████████████| 895 kB 34.7 MB/s 
     |████████████████████████████████| 596 kB 35.6 MB/s 
     |████████████████████████████████| 3.3 MB 36.8 MB/s 
     |████████████████████████████████| 59 kB 6.9 MB/s 
     |████████████████████████████████| 284 kB 28.4 MB/s 
     |████████████████████████████████| 321 kB 32.7 MB/s 
  Created wheel for emoji: filename=emoji-1.6.1-py3-none-any.whl size=169314 sha256=ad08f16c9e8ad4ec43937aacc2a6895a336bea8eda6dca11ee71802b744f711f
  Stored in directory: /root/.cache/pip/wheels/ea/5f/d3/03d313ddb3c2a1a427bb4690f1621eea60fe6f2a30cc95940f
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85433 sha256=48371fadc8d25680ce5d8527b3b043f25e14c7df90efbaa7bebfc96ab9cc4033
  Stored in directory: /root/.cache/pip/wheels/25/

In [ ]:
import pandas as pd
import numpy as np
import string
import regex as re
import contractions
import emoji
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
# from simpletransformers.classification import ClassificationModel
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
import pickle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def remove_emojis (text):
        return text.apply (lambda x: emoji.get_emoji_regexp ().sub (u'', x))


def expand_hashtags ( sentences):
        hashtag_pattern = re.compile (r'#([\p{L}0-9\_]+)')
        return sentences.apply (lambda x: re.sub (hashtag_pattern, lambda match: camel_case_split (match.group (1)), x))


def camel_case_split ( identifier):
        # link https://stackoverflow.com/questions/29916065/how-to-do-camelcase-split-in-python/29920015
        matches = re.finditer ('.+?(?:(?<=\p{Ll})(?=\p{Lu})|(?<=\p{Lu})(?=\p{Lu}\p{Ll})|[0-9]+|$)', identifier)
        return ' '.join ([m.group (0) for m in matches])


def remove_links(s):
    removeLinksPattern = re.compile(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+',re.DOTALL)
    tmp = re.sub(removeLinksPattern, ' ', s)#, flags = re.DOTALL)
    return tmp 


def remove_mentions(text):
    begining_mentions_pattern = re.compile (r"^(@[A-Za-z0-9\_]+\s?)+")
    middle_mentions_pattern = re.compile (r'(?<!\b)@([A-Za-z0-9\_]+)\b(?<!user)')
    txt = text.apply (lambda x: re.sub (begining_mentions_pattern, '', x))
    return txt


def remove_percentages(text):
    percents_pattern = re.compile(r"\b(\d+[\.,]?\d*|\d{2}[AP]M)%")
    #percents_pattern = re.compile(r"(\d+(\.\d+)?%)")
    return text.apply (lambda x: re.sub(percents_pattern, ' ', x))#   re.findall(percents_pattern, text)


def check_percentages(text):
    percents_pattern = re.compile(r"\b(\d+[\.,]?\d*|\d{2}[AP]M)%")
    #percents_pattern = re.compile(r"(\d+(\.\d+)?%)")
    return text.apply (lambda x: re.findall(percents_pattern,  x))#   re.findall(percents_pattern, text)


def remove_digits (text, replace = '[NUMERO]'):
        digits_pattern = re.compile (r"\b(\d+[\.,]?\d*|\d{2}[AP]M)\b")
        return text.apply (lambda x: re.sub (digits_pattern, replace, x))


def replace_keep_case (word, replacement, text):
    def func (match):
        g = match.group ()
        if g.islower (): return replacement.lower ()
        if g.istitle (): return replacement.title ()
        if g.isupper (): return replacement.upper ()
        return replacement
    return re.sub (word, func, text)


def remove_elongations (text):
        laughs_pattern = re.compile (r'(?i)\b(mua)?j[ja]+a?\b')
        # Remove laughs
        txt = text.apply (lambda x: re.sub (laughs_pattern, 'jajaja', x))

        
        # Remove exclamatory and interrogative
        for character in ['!', '¡', '?', '¿']:
            pattern = re.compile ('\\' + character + '{2,}')
            txt = txt.apply (lambda x: re.sub (pattern, character, x))
        
        # Add (.)\1{2,}|[aá]{2,}|[eé]{2,}|[ií]{2,}|[oó]{2,}|[uú]{2,})
        elongation_pattern = re.compile (r'(.)\1{2,}')
        # Remove letters longer than 2
        txt = txt.apply (lambda x: re.sub (elongation_pattern, r'\1', x))
        
        txt = txt.apply (lambda x: replace_keep_case (re.compile (r'(?i)[aá]{2,}'), "á", x))
        txt = txt.apply (lambda x: replace_keep_case (re.compile (r'(?i)[eé]{2,}'), "é", x))
        txt = txt.apply (lambda x: replace_keep_case (re.compile (r'(?i)[ií]{2,}'), "í", x))
        txt = txt.apply (lambda x: replace_keep_case (re.compile (r'(?i)[oó]{2,}'), "ó", x))
        txt = txt.apply (lambda x: replace_keep_case (re.compile (r'(?i)[uú]{2,}'), "ú", x))
        return txt


def remove_quotations (text):
        quotations_pattern = r'["“”\'«»‘’]'
        return text.replace (to_replace = quotations_pattern, value = '', regex = True)


def remove_punctuation (text):
    punctuation_pattern = re.compile ("[" + re.escape(string.punctuation) + "]")
    #print(punctuation_pattern)
    punc = text.apply (lambda x: re.findall(punctuation_pattern,  x))
    #print(punc)
    return text.apply (lambda x: re.sub (punctuation_pattern, '', x))

In [ ]:
def preprocessData(text):
  txt = expand_hashtags(text)
  txt = txt.apply(lambda x: remove_links(x))
  txt = remove_mentions(txt)
  txt = remove_percentages(txt)
  txt = remove_digits(txt)
  txt = remove_elongations(txt)
  txt = remove_emojis(txt)
  txt = txt.str.lower()
  txt = remove_quotations(txt)
  txt = remove_punctuation(txt)
  txt = txt.apply(lambda x: x.strip())
  return txt

In [ ]:
train_data = pd.read_csv("/content/drive/My Drive/sexism_detection/EXIST2021_training.tsv", sep="\t")
# train_data = train_data.drop(["test_case", "source", "language", "id"], axis = 1)

In [ ]:
train_data.text = preprocessData(train_data.text)

In [ ]:
train_data.loc[train_data['language'] == 'en'].head(5)

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,en,she calls herself antifeminazi how about shut ...,sexist,ideological-inequality
1,EXIST2021,2,twitter,en,now back to these women the brave and the beau...,non-sexist,non-sexist
2,EXIST2021,3,twitter,en,wow your skirt is very short what is its lengt...,sexist,objectification
3,EXIST2021,4,twitter,en,incredible beautifulbut i laughed so much whe...,non-sexist,non-sexist
4,EXIST2021,5,twitter,en,i find it extremely hard to believe that kelly...,non-sexist,non-sexist


In [ ]:
eng_train_data_text = list(train_data.loc[train_data['language'] == 'en'].text)
eng_train_data_text[:5]

['she calls herself antifeminazi how about shut the fucking up on your vile commentary on an elderly responsible citizen tu sach múch ghani báwribewdi hai bey',
 'now back to these women the brave and the beautiful clarecrawley and tayshia these bad ass babes are deserve so much credit for how this season has gone as a woman ive learned so much from them and fél more empowered to expect more in future relationships',
 'wow your skirt is very short what is its length numero inch or more',
 'incredible  beautifulbut i laughed so much when i read about you drifting in your whélchairi can just picture it',
 'i find it extremely hard to believe that kelly or a numero yr old mum would believe any of this so it makes me question what the fuck their game was especially as none of them have apologised or tried to explain amp their immediate response was to like twéts attacking the victims']

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [ ]:
device = "cuda:0" 
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-en-es")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-en-es").to(device) 

def translate_en_to_es(text_list): 
  # Tokenize the text
  batch = tokenizer(text_list, return_tensors="pt", padding=True).to(device)
                  
  # Make sure that the tokenized text does not exceed the maximum
  # allowed size of 512
  batch["input_ids"] = batch["input_ids"][:, :512]
  batch["attention_mask"] = batch["attention_mask"][:, :512]# Perform the translation and decode the output
  translation = model.generate(**batch)
  return tokenizer.batch_decode(translation, skip_special_tokens=True)

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

In [ ]:
translated_spanish = []
l = len(eng_train_data_text)
for i in range(0,l,30):
  translated_spanish += translate_en_to_es(eng_train_data_text[i:min(i+30,l)])

In [ ]:
len(translated_spanish)

3436

In [ ]:
with open("/content/drive/My Drive/sexism_detection/translated_spanish.pkl",'wb') as f:
  pickle.dump(translated_spanish,f)

In [ ]:
train_data_translated_to_spanish = train_data[train_data.language != 'es']
train_data_translated_to_spanish.text = translated_spanish
train_data_translated_to_spanish.replace('en', 'es',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
train_data_translated_to_spanish.head(5)

,test_case,id,source,language,text,task1,task2
0,EXIST2021,1,twitter,es,Ella se hace llamar antifeminazi ¿qué tal si c...,sexist,ideological-inequality
1,EXIST2021,2,twitter,es,Ahora de vuelta a estas mujeres la valiente y ...,non-sexist,non-sexist
2,EXIST2021,3,twitter,es,wow tu falda es muy corta ¿cuál es su longitud...,sexist,objectification
3,EXIST2021,4,twitter,es,"Increíble belleza, pero me reí tanto cuando le...",non-sexist,non-sexist
4,EXIST2021,5,twitter,es,Me resulta extremadamente difícil creer que Ke...,non-sexist,non-sexist


In [ ]:
train_data_spanish = pd.concat([train_data[train_data.language != 'en'], train_data_translated_to_spanish], ignore_index=True)
train_data_spanish.shape

(6977, 7)

In [ ]:
with open("/content/drive/My Drive/sexism_detection/combined_spanish_df.pkl",'wb') as f:
  pickle.dump(train_data_spanish,f)

In [ ]:
es_train_data_text = list(train_data.loc[train_data['language'] == 'es'].text)
es_train_data_text[:5]

['nadie te va a tratar tan bien como un hombre que te lo quiere meter por primera vez',
 'que rica putita obediente afortunado tu marido de tener una mujer como tú saludos',
 'yo lo hice a los numero años por la carretera libre a veracruz y ahora hay más carreteras veras que si puedes mujer',
 'las cosas q sueño son indicios de que yo enrealidad soy una lesbiana reprimida no tengo ninguna duda d esto',
 'pero a la niña le gustó desde que lo vió así que me le dije hola bien y tú ese día sólo podía ver un hombre guapo moreno como me gustan con una barba que rodeaba su boca tan perfecta y unos ojazos que wow hasta las manos le vi ajjaja por que si me fijo en eso']

In [ ]:
device = "cuda:0" 
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-es-en")
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-es-en").to(device) 

def translate_es_to_en(text_list): 
  # Tokenize the text
  batch = tokenizer(text_list, return_tensors="pt", padding=True).to(device)
                  
  # Make sure that the tokenized text does not exceed the maximum
  # allowed size of 512
  batch["input_ids"] = batch["input_ids"][:, :512]
  batch["attention_mask"] = batch["attention_mask"][:, :512]# Perform the translation and decode the output
  translation = model.generate(**batch)
  return tokenizer.batch_decode(translation, skip_special_tokens=True)

Downloading:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/807k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/783k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298M [00:00<?, ?B/s]

In [ ]:
translated_english = []
l = len(es_train_data_text)
for i in range(0,l,30):
  translated_english += translate_es_to_en(es_train_data_text[i:min(i+30,l)])

In [ ]:
len(translated_english)

3541

In [ ]:
with open("/content/drive/My Drive/sexism_detection/translated_english.pkl",'wb') as f:
  pickle.dump(translated_english,f)

In [ ]:
train_data_translated_to_english = train_data[train_data.language != 'en']
train_data_translated_to_english.text = translated_english
train_data_translated_to_english.replace('es', 'en',inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4389: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [ ]:
train_data_translated_to_english.head(5)

,test_case,id,source,language,text,task1,task2
3436,EXIST2021,3437,twitter,en,No one's gonna treat you as well as a man who ...,sexist,sexual-violence
3437,EXIST2021,3438,twitter,en,What a rich obedient slut. Lucky your husband ...,sexist,stereotyping-dominance
3438,EXIST2021,3439,twitter,en,I did it to the number of years on the free ro...,non-sexist,non-sexist
3439,EXIST2021,3440,twitter,en,I dream things are indications that I'm actual...,non-sexist,non-sexist
3440,EXIST2021,3441,twitter,en,But the girl liked it since she saw it so I sa...,non-sexist,non-sexist


In [ ]:
train_data_english = pd.concat([train_data[train_data.language != 'es'], train_data_translated_to_english], ignore_index=True)
train_data_english.shape

(6977, 7)

In [ ]:
with open("/content/drive/My Drive/sexism_detection/combined_english_df.pkl",'wb') as f:
  pickle.dump(train_data_english,f)

In [ ]:
augmented_train_data = pd.concat([train_data_english, train_data_spanish], ignore_index=True)
augmented_train_data.shape

(13954, 7)

In [ ]:
with open("/content/drive/My Drive/sexism_detection/augmented_train_data_df.pkl",'wb') as f:
  pickle.dump(augmented_train_data,f)